In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.models import load_model

def model_load(model=''):
  return load_model(model)

In [ ]:
cnn_lstm=model_load('/content/drive/MyDrive/CSE-400(Project Thesis)/hash_cnn_lstm.h5')

In [ ]:
!pip install bnlp-toolkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 9.3 MB/s 
     |████████████████████████████████| 23.9 MB 1.1 MB/s 
     |████████████████████████████████| 965 kB 62.9 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import re
from bnlp.corpus import stopwords

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
def preprocess(data):
    """This function is for preprocessing of the news contents. It removes punctuations, English characters and both of Bangla
    and English numerals. It tokenizes all the words and also removes stopwords.
    
    Args: 
        The Entire Dataframe
    Returns:
        Preprocessed news corpuses in a list of lists
    """
    corpus=[]
    for i in range(0, len(data)):
        x=re.sub('[^\u0980-\u09FF]',' ',data['content'][i])
        x=re.sub('[a-zA-Z0-9]+', ' ', x)
        x=re.sub('[০১২৩৪৫৬৭৮৯]+', ' ', x)
        x=x.split()
        x=[w for w in x if w not in set(stopwords)]
        x=' '.join(x)
        corpus.append(x)
    
    return corpus

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
sns.set()

In [ ]:
label_auth=pd.read_csv(r'/content/drive/MyDrive/AutoEncoders/LabeledAuthentic-7K.csv') #, nrows=3067)
label_fake=pd.read_csv(r'/content/drive/MyDrive/AutoEncoders/LabeledFake-1K.csv') #, nrows=995)
label_auth.drop(['source','relation'], axis=1, inplace=True)
label_fake.drop(['source','relation','F-type'], axis=1, inplace=True)
df=pd.concat([label_auth, label_fake], axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df['label']=np.where(df['label']==0.0, 0, 1)
df.tail()
df=df.sample(frac=1).reset_index(drop=True)
print(df.shape)
df.head()

(8501, 7)


,articleID,domain,date,category,headline,content,label
0,5442,bd-pratidin.com,2018-09-20 08:53:16,Miscellaneous,"অদ্ভুত মাছের সন্ধান, হাতে নিলেই গলে যায়! (ভিডিও)",আটাকামা স্নেইল ফিশ। এদের শরীর অদ্ভুত স্বচ্ছ। শ...,1
1,3361,bd24live.com,2018-09-20 20:15:59,National,ট্রাম্পের সঙ্গে ‘যৌনসম্পর্ক’ কেমন ছিল জানালেন ...,মার্কিন প্রেসিডেন্ট নির্বাচিত হওয়ার পর ডোনাল্ড...,1
2,3600,jagonews24.com,2018-09-19 14:19:00,Politics,জামায়াতের সঙ্গ ত্যাগের দাবি ‘উপভোগ’ করছে বিএনপি,সরকারবিরোধী বৃহত্তর ঐক্য গঠনে জামায়াতের সঙ্গ ত...,1
3,898,channeldhaka.news,2019-05-03T16:00:13+00:00,National,ঘূর্ণিঝড় ফণীর আঘাতে ঢাকা শহর টিকবে তো?,মানুষের স্পর্ধা দেখে বেশ অবাকই লাগে। সুপার সাই...,0
4,137,independent24.com,2018-09-24 16:27:54,National,ফাইনালের আশা টিকিয়ে রাখল টাইগাররা,স্বস্তির জয়ে এশিয়া কাপে ফাইনালের আশা জিইয়ে রাখ...,1


In [ ]:
print('The input text is:\n')
df['content'][4]

The input text is:



'স্বস্তির জয়ে এশিয়া কাপে ফাইনালের আশা জিইয়ে রাখল টাইগাররা। শ্বাসরুদ্ধকর ম্যাচে আফগানিস্তানকে ৩ রানে হারালো বাংলাদেশ। আফগানদের আড়াইশো রানের টার্গেট দেয় টাইগাররা। শেষ ওভারে দরকার ছিলো ৮ রান। শেষ পর্যন্ত, মুস্তাফিজের কাটারেই বাজিমাত। ২৪৬ রানে থামে আফগানদের ইনিংস। শেষ ওভারের দরকার ৮ রান। টি-টোয়েন্টি ধামাকার যুগে এই রান ডিফেন্ড করা অসম্ভবের মতো। তবে মুস্তাফিজের হাত বল বলেই কিছুটা আশা ছিল। প্রথম বলে দুই রান নিয়ে জয় অনেকটা সময়ের অপেক্ষা বানায় আফগানরা। তবে ফিজের জাদু তখনই শুরু। রাশিদ খানকে ফিরিয়ে, দুই ডট দিয়ে অবিশ্বাস জয় এনে দেন মুস্তাফিজ। আফগানদের বিপক্ষে প্রতিশোধের সাথে এশিয়া কাপের ফাইনালে আশা জেগে রাখল বাংলাদেশ। উইকেট, কন্ডিশন আর টানা খেলার ধকলের মধ্যে ২৫০ রান চেজ করার সব সময় কষ্ট সাধ্য। সেই কাজ আফগানদরে আরও কঠিন হয়ে যায় দ্রুত দুই উইকেট হারালে। বাংলাদেশকে আপারহ্যান্ডে রাখেন মুস্তাফিজ ও সাকিবের করা রান আউট। তবে এশিয়া কাপে দারুন ফর্মে থাকা শাহিদি, ফর্ম ধরে রাখলেন। চাপ কমিয়ে আসতে আসতে আফগানদের লক্ষ্যে দিকে নিতে থাকেন এই ব্যাটসম্যান।শেহজাদের সাথে গড়েন ৬৩ রানের জুটি। শেহজাদকে ফিরিয়ে ব্রেকেথ্রু এ

In [ ]:
corpus=preprocess(df)

In [ ]:
print('The cleaned text is:\n')
corpus[4]

The cleaned text is:



'স্বস্তির জয়ে এশিয়া কাপে ফাইনালের আশা জিইয়ে রাখল টাইগাররা শ্বাসরুদ্ধকর ম্যাচে আফগানিস্তানকে রানে হারালো বাংলাদেশ আফগানদের আড়াইশো রানের টার্গেট দেয় টাইগাররা শেষ ওভারে দরকার ছিলো রান শেষ মুস্তাফিজের কাটারেই বাজিমাত রানে থামে আফগানদের ইনিংস শেষ ওভারের দরকার রান টোয়েন্টি ধামাকার যুগে রান ডিফেন্ড অসম্ভবের মুস্তাফিজের হাত বল বলেই কিছুটা আশা রান জয় অনেকটা সময়ের অপেক্ষা বানায় আফগানরা ফিজের জাদু তখনই রাশিদ খানকে ফিরিয়ে ডট দিয়ে অবিশ্বাস জয় এনে মুস্তাফিজ আফগানদের বিপক্ষে প্রতিশোধের সাথে এশিয়া কাপের ফাইনালে আশা জেগে রাখল বাংলাদেশ উইকেট কন্ডিশন টানা খেলার ধকলের রান চেজ সময় কষ্ট সাধ্য আফগানদরে কঠিন হয়ে যায় দ্রুত উইকেট হারালে বাংলাদেশকে আপারহ্যান্ডে রাখেন মুস্তাফিজ সাকিবের রান আউট এশিয়া কাপে দারুন ফর্মে শাহিদি ফর্ম রাখলেন চাপ কমিয়ে আসতে আসতে আফগানদের লক্ষ্যে ব্যাটসম্যান শেহজাদের সাথে গড়েন রানের জুটি শেহজাদকে ফিরিয়ে ব্রেকেথ্রু এনে মাহমুদুল্লাহ এরপর আবারও দেখেশুনে খেলে রানের পার্টনারশীপ গড়েন আসগর শাহিদি রানে শাহিদি আসগর রানে ফিরলে ম্যাচে ফেরে বাংলাদেশ শেষ ভয় জাগান নাবী শেনওয়ারী টস জিতে ব্যাট নেমে আবারও

In [ ]:
y=df['label']

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.sequence import pad_sequences

In [ ]:
voc_size=150000
hashing_corpus=[hashing_trick(words, n=voc_size, hash_function='md5', lower=False) for words in corpus]

In [ ]:
print('The input vector after hashing is:\n')
print(hashing_corpus[4][0:15])

The input vector after hashing is:

[139676, 7878, 108521, 84537, 95430, 115511, 144496, 122648, 50915, 103723, 115856, 139921, 94272, 33540, 101977]


In [ ]:
sent_length=300
embedded_docs=pad_sequences(hashing_corpus, maxlen=sent_length, truncating='post')

In [ ]:
print('The input embedded document is:\n')
print(embedded_docs[4][60:85])

The input embedded document is:

[     0      0      0      0      0      0      0      0      0      0
 139676   7878 108521  84537  95430 115511 144496 122648  50915 103723
 115856 139921  94272  33540 101977]


In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_final.shape, y_final.shape

((8501, 300), (8501,))

In [ ]:
cnn_lstm_pred=cnn_lstm.predict(X_final)
cnn_lstm_pred=cnn_lstm_pred.round()

y=['Fake' if x==0 else 'Real' for x in y]
cnn_lstm_pred=['Fake' if x==0 else 'Real' for x in cnn_lstm_pred]

print(f'Actual Outcome ---> {y[4]} \n')
print(f'Prediction Outcome ---> {cnn_lstm_pred[4]}')

Actual Outcome ---> Real 

Prediction Outcome ---> Real


In [ ]:
print('The input text is:\n')
df['content'][3]

The input text is:



'মানুষের স্পর্ধা দেখে বেশ অবাকই লাগে। সুপার সাইক্লোন “ফণী” এর ভেলোসিটি এরাউন্ড ২০০ কিমি/ঘন্টা। ঢাকা শহরে যে লোকাল বাসগুলো চলে তার গতিবেগ মাত্র ২০ কিমি/ ঘন্টা এর আশেপাশে। হাইওয়েতে গাড়িগুলো সর্বোচ্চ ১২০ কিমি/ঘন্টা বা এর একটু উপরে হয়তো যায়। তাতেই মনে কাঁপন ধরে যায়, এই বুঝি আরেকটা এক্সিডেন্ট হলো।ভাবুন তো, ২০০ বা তারও বেশি বেগের বাতাস অনেকক্ষণ ধরে প্রবাহিত হচ্ছে আর সাথে তুমুল বৃষ্টি। পুরো বাংলাদেশে মিনিট কয়েক এর মতোও যদি বয়ে যায়, বিশ্বমানচিত্রে এর অস্তিত্ব থাকবে কিনা সন্দেহ আছে। সেখানে বাংলাদেশীরা রোহিঙ্গাদের নিয়ে মজা করছে। এই ঝড়ে যেনো তারা ভেসে যায়, উড়ে গিয়ে যেনো আবার মিয়ানমারে পরে। ঝড় আসলে কি তা জাত-পাত দেখে আসবে? আবার খবরের কাগজে লিখেছে “ফণী” এর মোকাবেলায় প্রস্তুত বাংলাদেশ। যেনো ঢাল তলোয়ার নিয়ে যুদ্ধে নামছে। কিছু মানুষকে নিরাপদ আশ্রয়ে সরিয়ে নেওয়া ছাড়া আর কী করতে পারে মানুষ? সেই তথাকথিত উন্নতি প্রযুক্তি দিয়েও আমেরিকা অগ্ন্যুৎপাত থেকে বাঁচতে পারে না।নেপালের ভূমিকম্পও তো বেশিদিন আগের কথা নয়। মানুষ কী নিয়ে এত অহংকার করে? তারা কি আদ-সামুদ জাতিকে দেখে না? তাদের অস্তিত্

In [ ]:
print('The cleaned text is:\n')
corpus[3]

The cleaned text is:



'মানুষের স্পর্ধা অবাকই লাগে সুপার সাইক্লোন ফণী ভেলোসিটি এরাউন্ড কিমি ঘন্টা ঢাকা শহরে লোকাল বাসগুলো গতিবেগ কিমি ঘন্টা আশেপাশে হাইওয়েতে গাড়িগুলো সর্বোচ্চ কিমি ঘন্টা একটু তাতেই কাঁপন বুঝি আরেকটা এক্সিডেন্ট ভাবুন তারও বেগের বাতাস অনেকক্ষণ প্রবাহিত সাথে তুমুল বৃষ্টি পুরো বাংলাদেশে মিনিট মতোও বয়ে বিশ্বমানচিত্রে অস্তিত্ব কিনা সন্দেহ বাংলাদেশীরা রোহিঙ্গাদের মজা ঝড়ে যেনো ভেসে উড়ে যেনো মিয়ানমারে ঝড় আসলে জাত পাত আসবে খবরের কাগজে লিখেছে ফণী মোকাবেলায় প্রস্তুত বাংলাদেশ যেনো ঢাল তলোয়ার যুদ্ধে নামছে মানুষকে নিরাপদ আশ্রয়ে সরিয়ে মানুষ তথাকথিত উন্নতি প্রযুক্তি দিয়েও আমেরিকা অগ্ন্যুৎপাত বাঁচতে নেপালের ভূমিকম্পও বেশিদিন আগের কথা মানুষ অহংকার আদ সামুদ জাতিকে অস্তিত্ব ছিলো বোঝার উপায় আল্লাহর আযাব সেভাবেই গ্রাস নূহ আলাইহিস সালামের কওমেরা কোথায় তুলনায় বাঙ্গালীদের প্রভাব প্রতিপত্তি কোনোদিকেই তুলনা এক পাল্লায় রাখার মতোও এরপরেও পাপাচারের ঔদ্ধত্য সীমা ছাড়িয়ে আল্লাহ আযাব হিফাজত করুন লেখা মিসবাহ মাহীন'

In [ ]:
print('The input vector after hashing is:\n')
print(hashing_corpus[3][0:15])

The input vector after hashing is:

[120984, 139819, 56452, 86003, 4568, 81530, 110717, 10283, 142836, 108628, 119565, 93397, 147327, 119048, 90979]


In [ ]:
print('The input embedded document is:\n')
print(embedded_docs[3][165:200])

The input embedded document is:

[     0      0      0      0 120984 139819  56452  86003   4568  81530
 110717  10283 142836 108628 119565  93397 147327 119048  90979 134292
 108628 119565  12597  10216  97473  33773 108628 119565 147653 114324
  61721  51771 101436  20273  23139]


In [ ]:
print(f'Actual Outcome ---> {y[3]} \n')
print(f'Prediction Outcome ---> {cnn_lstm_pred[3]}')

Actual Outcome ---> Fake 

Prediction Outcome ---> Fake
